In [19]:
#Analysis 2, only jump risk 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import fsolve

r=0.05 #risk free rate #Liu and Pan
eta= 4 #risk premium of diffusive risk #Liu and Pan
kappa= 5 #speed of mean reversion for V_t, >0 #Liu and Pan
nu_bar= 0.15**2 #long run volatility, >0 #Liu and Pan
sigma= 0 #volatility coefficient, >=0  #Liu and Pan
rho=-0.4 #correlation between diffusive and volatility shocks, 1>rho>-1 #Liu and Pan
gamma=3 #Liu and Pan (2003)
xi=-6
mu=-0.1 #start value 10%
V=nu_bar
T = 5 # Total time
S0 = 100  # Initial stock price
V0 = nu_bar  # Initial variance
pi_0 = 1  # Parametric pricing kernel #Liu and Pan


In [20]:
# Define functions for A(y) and B(y) as in the appendix
def B(y, a, q, b, t):
    term1 = (a * (1 - np.exp(-q * t))) / (2 * q - (q + b) * (1 - np.exp(-q * t)))
    return -term1

def A(y, kappa_star, v_bar_star, sigma, q, b, tau):
    #term1 =  #kappa_star * v_bar_star / sigma**2
    term2 = (q+b)*tau + 2*np.log(1 - (q + b) / (2 * q) * (1 - np.exp(-q*tau))) 
    return  - term2 #deleted term 1

# Characteristic function integrands for P1 and P2
def P1_integrand(u, S, K, V, r, tau, sigma, rho, kappa_star, lambda_q, mu):
    y = 1 - 1j * u

    a = y*(1-y)-2*lambda_q*(np.exp(y)*(1+mu)-1-y*mu)
    b = sigma * rho * y - kappa_star
    q = np.sqrt(b**2 + a*sigma**2)
    
    B_val = B(y, a, q, b, T-tau)
    A_val = A(y, kappa_star, v_bar_star, sigma, q, b, tau)
    exponent = A_val + B_val * V + 1j * u * (np.log(K/S) + r * tau)
    return np.imag(np.exp(exponent)) / u

def P2_integrand(u, S, K, V, r, tau, sigma, rho, kappa_star, lambda_q, mu):
    y = -1j * u

    a = y*(1-y)-2*lambda_q*(np.exp(y)*(1+mu)-1-y*mu)
    b = sigma * rho * y - kappa_star
    q = np.sqrt(b**2 + a*sigma**2)

    B_val = B(y, a, q, b, T-tau)
    A_val = A(y, kappa_star, v_bar_star, sigma, q, b, tau)
    exponent = A_val + B_val * V + 1j * u * (np.log(K/S) + r * tau)
    return np.imag(np.exp(exponent)) / u

# Compute P1 and P2
def compute_P1_P2(S, K, V, r, tau, sigma, rho, kappa_star, lambda_q, mu):
    P1 = 0.5 - (1 / np.pi) * quad(P1_integrand, 0, 10000, args=(S, K, V, r, tau, sigma, rho, kappa_star, lambda_q, mu))[0]
    P2 = 0.5 - (1 / np.pi) * quad(P2_integrand, 0, 10000, args=(S, K, V, r, tau, sigma, rho, kappa_star, lambda_q, mu))[0]
    return P1, P2

# European option pricing and delta calculation
def european_option_price_and_delta(S, K, V, r, tau, option_type="call"):
    P1, P2 = compute_P1_P2(S, K, V, r, tau, sigma, rho, kappa_star, lambda_q, mu)
    if option_type == "call":
        price = S * P1 - np.exp(-r * tau) * K * P2
        delta = P1
    elif option_type == "put":
        price = np.exp(-r * tau) * K * (1 - P2) - S * (1 - P1)
        delta = P1-1
    return price, delta

# Calculate delta-neutral straddle
def calculate_straddle_delta(S, K, V, r, tau):
    _, delta_call = european_option_price_and_delta(S, K, V, r, tau, option_type="call")
    _, delta_put = european_option_price_and_delta(S, K, V, r, tau, option_type="put")
    straddle_delta = delta_call + delta_put
    return straddle_delta


# Vega for European option (partial derivative w.r.t. volatility)
def european_option_vega(S, K, V, r, tau, option_type="call"):
    epsilon = 1e-5  # Small change in V for numerical differentiation

    P1_high, P2_high = compute_P1_P2(S, K, V + epsilon, r, tau, sigma, rho, kappa_star, lambda_q, mu)
    P1_low, P2_low = compute_P1_P2(S, K, V - epsilon, r, tau, sigma, rho, kappa_star, lambda_q, mu)
    
    if option_type == "call":
        price_high = S * P1_high - np.exp(-r * tau) * K * P2_high
        price_low = S * P1_low - np.exp(-r * tau) * K * P2_low
    elif option_type == "put":
        price_high = np.exp(-r * tau) * K * (1 - P2_high) - S * (1 - P1_high)
        price_low = np.exp(-r * tau) * K * (1 - P2_low) - S * (1 - P1_low)
    
    # Calculate vega using central difference approximation
    vega = (price_high -price_low) / (2 * epsilon)
    return vega

# Calculate vega-neutral straddle
def calculate_straddle_vega(S, K, V, r, tau):
    vega_call = european_option_vega(S, K, V, r, tau, option_type="call")
    vega_put = european_option_vega(S, K, V, r, tau, option_type="put")
    straddle_vega = vega_call + vega_put
    return straddle_vega

# Compute k1, k2, and delta based on the parameters
def compute_k1_k2_delta(gamma, eta, xi, rho, sigma, lambda_n, lambda_q, mu, kappa):
    delta = ((1 - gamma) / gamma**2) * (eta**2 + xi**2) + 2 * lambda_q * ((lambda_n / lambda_q)**(1/gamma) + (1 / gamma) * (1 - lambda_n / lambda_q)-1)
    k1 = kappa - (1 - gamma) / gamma * (eta * rho + xi * np.sqrt(1 - rho**2)) * sigma
    k2 = np.sqrt(k1**2 - delta * sigma**2)
    return k1, k2, delta

def phi_psi(gamma, eta, rho, xi, sigma, k1, k2, delta, g_s_2, g_v_2, delta_g_2, S_paths, O_path_2, lambda_n, lambda_q):
    psi_star=((delta_g_2)/(mu*O_path_2)-(g_s_2*S_paths)/(O_path_2))**(-1)*((1/mu)*((lambda_n/lambda_q)**(1/gamma)-1)-eta/gamma)
    phi_star=eta/gamma-psi_star*((g_s_2)*S_paths/(O_path_2))

    return phi_star, psi_star

def phi_no_op(eta, gamma, mu, lambda_ratio, phi_guess):
    
    phi_no_op_star=(eta/gamma)+(gamma*mu)/gamma((1+phi_guess)**(-gamma)-lambda_ratio)

    return phi_no_op_star

# Compute the time-dependent function H(tau)
def H(tau, k1, k2, delta):
    denominator = 2 * k2 + (k1 + k2) * (np.exp(k2 * tau) - 1)
    return (np.exp(k2*tau)-1) / denominator*delta

def solve_phi(eta, gamma, lambda_n, lambda_q, mu, initial_guess=0):
    def equation(phi):
        left_side = phi
        right_side = (eta / gamma) + (lambda_n * mu / gamma) * (
            (1 + mu * phi)**(-gamma) - lambda_q/lambda_n)
        return left_side - right_side

    # Solve the equation numerically
    phi_no_op_star = fsolve(equation, initial_guess)

    return phi_no_op_star

In [21]:
# Define parameter values for looping
gamma_vals = [1, 3, 5, 10]
lambda_ratios = [1, 2, 5]
jump_scenarios = [
    {"mu": -0.10, "frequency": "every 10 yrs"},
    {"mu": -0.25, "frequency": "every 50 yrs"},
    {"mu": -0.50, "frequency": "every 200 yrs"},]  # (mu, time in years)

# Placeholder function to calculate weights
def calculate_weights(gamma, lambda_ratio, mu):
    # Replace these with actual calculations
    phi_star = 0  # Placeholder value, replace with actual calculation
    psi_star = 0  # Placeholder value, replace with actual calculation
    return phi_star, psi_star

# Collect data in a list
data = []

#We create 3 loops, mu, gamma, lambda_ratios

for scenario in jump_scenarios:
    mu = scenario["mu"]
    frequency = scenario["frequency"]
    
    for gamma in gamma_vals:
        for lambda_ratio in lambda_ratios:

            phi_values=[]
            psi_values=[]
            phi_no_op_values=[]


        #Now we insert the actual model: 
        #-------------------------------------------------------#

            n_steps=1000 # Number of steps
            dt = T/n_steps  # Time step
            t = np.linspace(0, T, n_steps+1)

            #define particular parameters
            lambda_n=0.2
            lambda_q=lambda_ratio*lambda_n

            #equity risk premium (se Liu and Pan ligning 4), kept constant at 8%, adjust eta accordingly
            #0.08=eta*nu_bar+mu*(lambda_n-lambda_q)*nu_bar
            eta=(0.08-mu*(lambda_n-lambda_q)*nu_bar)/nu_bar

            # Standard Brownian motion for B_t and _t and a jump-pro
            dBt = np.zeros(n_steps+1)
            dZt = np.zeros(n_steps+1)
                    
            for i in range(1, n_steps+1):
                dBt[i] = dBt[i-1] + np.sqrt(dt) * np.random.normal()
                dZt[i] = dZt[i-1] + np.sqrt(dt) * np.random.normal()

            #We create a pure jump process N_t
            #lambda_j = 0.5  # Rate of the Poisson process (number of jumps per unit time)

            if mu==-0.1: 
                lambda_j=T/10
            if mu==-0.25: 
                lambda_j=T/50
            if mu==-0.5: 
                lambda_j=T/200

            # Initialize the jump process
            dNt = np.zeros(n_steps + 1)
            jump_times = np.cumsum(np.random.exponential(scale=1/lambda_j, size=1000))  # Generate jump times

            # Ensure the jump times do not exceed the total time
            jump_times = jump_times[jump_times <= T]

            # Plot the jump process
            for t in range(1, n_steps + 1):
                dNt[t] = dNt[t-1] + np.sum((jump_times >= (t-1) * dt) & (jump_times < t * dt))
                    
            # Initialize arrays for the simulation
            S_paths = np.zeros(n_steps)
            V_paths = np.zeros(n_steps)
            S_paths[0] = S0
            V_paths[0] = V0

            # Initialize arrays for the simulation
            pi_paths = np.zeros(n_steps)
            pi_paths[0] = pi_0

            for t in range(1, n_steps):

                # Stock price process (S_t)
                S_paths[t] = S_paths[t-1] +(r + eta * V + mu * (lambda_n - lambda_q) * V) * S_paths[t-1]*dt + \
                    np.sqrt(V)* S_paths[t-1] * (dBt[t]-dBt[t-1]) +mu* S_paths[t-1]*((dNt[t]-dNt[t-1]) - lambda_n * V * dt) \

                #Pricing kernel process
                pi_paths[t]=pi_paths[t-1]*(1-(r*dt+eta*np.sqrt(V)*(dBt[t]-dBt[t-1]))+xi*np.sqrt(V)*(dZt[t]-dZt[t-1]))+(lambda_q/lambda_n-1)*pi_paths[t-1]*((dNt[t]-dNt[t-1])-lambda_n*V*dt)

            tau = 1/12         # Time to maturity (in years)

            # Additional terms for B and A as per Liu and Pan
            kappa_star = kappa - sigma * (rho*eta + np.sqrt(1-rho)*xi )
            v_bar_star = kappa * nu_bar / kappa_star

            #We create a loop that calculates delta and vega for the straddle for different prices $S_t$

            # Initialize sensitivities
            g_s_1 = np.zeros(n_steps)
            g_v_1 = np.zeros(n_steps)
            price_1=np.zeros(n_steps)
            price_jump_1=np.zeros(n_steps)
            delta_g_1 = np.zeros(n_steps)

            g_s_2 = np.zeros(n_steps)
            g_v_2 = np.zeros(n_steps)
            price_2=np.zeros(n_steps)
            price_jump_2=np.zeros(n_steps)
            delta_g_2 = np.zeros(n_steps)

            g_s_1[0] = 0.5
            g_v_1[0] = 0.3
            g_s_2[0] = -0.5
            g_v_2[0] = 0.3
            delta_g_1[0] = 0
            delta_g_2[0] = 0

            for t in range(1, n_steps):
                # Calculate delta and vega for the call and put using finite differences. Note that the strike is set such that the put is 5% OTM
                price_1[t], g_s_1[t] = european_option_price_and_delta(S_paths[t],S_paths[t]*(1-0.05), V, r, tau, option_type="call")
                g_v_1[t] = european_option_vega(S_paths[t], S_paths[t]*(1-0.05), V, r, tau, option_type="call")
                price_2[t], g_s_2[t] = european_option_price_and_delta(S_paths[t], S_paths[t]*(1-0.05), V, r, tau, option_type="put")
                g_v_2[t] = european_option_vega(S_paths[t], S_paths[t]*(1-0.05), V, r, tau, option_type="put")

                price_jump_1[t], _ = european_option_price_and_delta(S_paths[t]*(1+mu), S_paths[t]*(1-0.05), V, r, tau, option_type="call")
                price_jump_2[t], _ = european_option_price_and_delta(S_paths[t]*(1+mu), S_paths[t]*(1-0.05), V, r, tau, option_type="put")

                # Calculate jump deltas for call and put
                delta_g_1[t] = price_jump_1[t]-price_1[t]
                delta_g_2[t] = price_jump_2[t]-price_2[t]

                #rather than just looking at one point in time we, average the weights

                # Compute k1, k2, and delta
                k1, k2, delta = compute_k1_k2_delta(gamma, eta, xi, rho, sigma, lambda_n, lambda_q, mu, kappa)     
                
                phi, psi = phi_psi(gamma, eta, rho, xi, sigma, k1, k2, delta, g_s_2[t], g_v_2[t], delta_g_2[t], S_paths[t], price_2[t], lambda_n, lambda_q)
                phi_no_op_value = solve_phi(eta, gamma, lambda_n, lambda_q, mu, initial_guess=0)

                phi_values.append(phi)
                psi_values.append(psi)
                phi_no_op_values.append(phi_no_op_value) 


            phi=np.mean(phi_values)
            psi=np.mean(psi_values)
            phi_no_op=np.mean(phi_no_op_values)
            
            #print(f"for gamma:{gamma} lambdaratio:{lambda_ratio} mu:{mu} we have, phi_no:{phi_no_op} phi:{phi} psi{psi}")
        #-------------------------------------------------------#

        # Add row to the table data
            data.append({
                "gamma": gamma,
                "lambda_ratio": lambda_ratio,
                "mu": f"{mu*100:.0f}%",  # Convert to percentage for display
                "frequency": frequency,
                "phi_stock_only": f"{phi_no_op:.2f}",
                "phi": f"{phi:.3f}",
                "psi": f"{psi:.3f}", 
            })

            

In [22]:
# Convert list to a DataFrame
df = pd.DataFrame(data)
 
# Format DataFrame for a more readable table output
df_table = df.pivot_table(index=["gamma", "lambda_ratio"], columns=["mu", "frequency"],
                          values=[ "phi_stock_only","phi", "psi"],
                          aggfunc="first")

# Reorder columns so phi_stock_only appears first
ordered_columns = ["phi_stock_only", "phi", "psi"]

# Reorder and sort the multi-index columns
df_table = df_table[ordered_columns]

import pandas.io.formats.style as style
df_table_style = df_table.style.set_table_styles([
    {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
]).set_caption("Table 1: Optimal strategies with/without options")
 
# Display the table
df_table_style
